# Experiments Using The SAFE Protocol

### Introduction

This notebook includes the experiments conducted using the SAFE Protocol. The goal was to assess the suitability of the SAFE Protocol for meeting the privacy and utility constraints outlined in the introduction of the report, as well as consider possible extensions.

The notebook has the following contents:
- Toy Implementation Of SAFE (How SAFE works)
- Evaluation of SAFE's Robustness
- Making SAFE more robust.

[make this a proper TOC with links]: # (add links to the above)

The Toy implementation of SAFE is merely illustrative-- mock data of a small size will be used to illustrate the basic mechanics of the SAFE protocol.

### Setting Up Toy Implementation

To illustrate how the SAFE Protocol works, we will mock the data of 5 users who contribute feature vectors. 

#### Set Up Raw Vector Creation

First, let's create the raw feature vectors for each user. These vectors represent the raw data the SAFE protocol wants to keep private whilst allowing for useful computation over.

[add more explanation]: # (explain what the vectors are)

In [1]:

# SET UP - Create raw feature vectors 

import torch
import numpy as np
    # for reproducibility
torch.manual_seed(0)

    # CONSTANTS for a run of the Protocol
NUMBER_OF_USERS = 3
NUMBER_OF_FEATURES_PER_USER_VECTOR = 2
VECTOR_SHAPE = (NUMBER_OF_FEATURES_PER_USER_VECTOR, 1)
# TODO: actually, VECTOR_SHAPE is redundant. That wasn't the source of the error. But, anyways.
 
raw_feature_vectors = {}

for i in range(NUMBER_OF_USERS):
    raw_feature_vectors[i] = torch.rand(VECTOR_SHAPE)
    print(f"Here are is the feature vector for user {i}: ")
    print(raw_feature_vectors[i])
    print("\n")

Here are is the feature vector for user 0: 
tensor([[0.4963],
        [0.7682]])


Here are is the feature vector for user 1: 
tensor([[0.0885],
        [0.1320]])


Here are is the feature vector for user 2: 
tensor([[0.3074],
        [0.6341]])




### SHARE CREATION AND DISTRIBUTION

Next, each user will generate (N-1) shares from the interval [-D, D], where N is the number of users. 

Each share is a K-dimensional vector, where K is the number of features each user's raw feature vector has.

Each user uses these N-1 shares to generate their own Nth share. 

Then, they will distribute the N-1 shares to the other N-1 users, whilst keeping their Nth share.

In [2]:
# SHARE CREATION

import random
from token import NUMBER
from unicodedata import name

from numpy import shares_memory 
random.seed(0)

D_VALUE = 10.0 # defining the interval from which users drawn random share vectors.

print("Generating the shares for each user \n")
each_users_shares_to_distribute = {} # mapping from i to list of tensors.
each_users_Nth_share = {} # mapping from i to 4-D tensor
for user in range(NUMBER_OF_USERS):
    # Each user first generates N-1 shares : a share to be sent to every other user.
    each_users_shares_to_distribute[user] = []
    for other_user in range(NUMBER_OF_USERS - 1):
        share_to_distribute = [random.uniform(-D_VALUE, D_VALUE) for i in range(NUMBER_OF_FEATURES_PER_USER_VECTOR)]
        share_to_distribute = torch.reshape(torch.tensor(share_to_distribute), VECTOR_SHAPE)
        each_users_shares_to_distribute[user].append(share_to_distribute)
    # Then, each user calculates their own Nth share from the N-1 Shares the draw from the interval
    sum_of_shares = torch.zeros(VECTOR_SHAPE)
    for share in each_users_shares_to_distribute[user]:
        sum_of_shares.add_(share)
    # print(f"Here is user_{user}'s sum of shares: {sum_of_shares}")
    each_users_Nth_share[user] = raw_feature_vectors[user] - sum_of_shares 
"""
TODO:
Not sure whether there's a better way to generate the shares. The calls to the random.uniform() method for each share vector value seems sub-optimal but I can't quite find an 'off-the-shelf' method that generates the matrix/dictionary more easily. This triple nested loop is not computationally efficient. Is commenting on the algorithm's time complexity worth doing?

TODO: 
Also not sure whether the 'random' provided by Python is sufficiently random. Or whether the presence of a 'seed' value complicates things.
"""

print("Let's take a look at shares each user will distribute:\n")
for user in range(NUMBER_OF_USERS):
    print(f"User {user}: \n")
    for share in each_users_shares_to_distribute[user]:
        print(share)
    print("\n")    
    print("And here's their Nth share:\n")
    print(each_users_Nth_share[user])
    print("\n")    


Generating the shares for each user 

Let's take a look at shares each user will distribute:

User 0: 

tensor([[6.8884],
        [5.1591]])
tensor([[-1.5886],
        [-4.8217]])


And here's their Nth share:

tensor([[-4.8036],
        [ 0.4308]])


User 1: 

tensor([[ 0.2255],
        [-1.9013]])
tensor([[ 5.6760],
        [-3.9337]])


And here's their Nth share:

tensor([[-5.8130],
        [ 5.9671]])


User 2: 

tensor([[-0.4681],
        [ 1.6676]])
tensor([[8.1623],
        [0.0937]])


And here's their Nth share:

tensor([[-7.3868],
        [-1.1273]])




Now, let's distribute the shares among the users.

In [3]:
# SHARE Distribution

print("Distributing shares...\n")
each_users_received_shares = {}
# set up the map
for i in range(NUMBER_OF_USERS):
    each_users_received_shares[i] = []
# fill the map
for user in range(NUMBER_OF_USERS):
    other_user_ids = [ind for ind in range(NUMBER_OF_USERS) if ind != user]
    # get the shares for user_0 to send to other users
    shares_to_send = [share for share in each_users_shares_to_distribute[user]]
    for other_user, share_to_send in zip(other_user_ids, shares_to_send):
        each_users_received_shares[other_user].append(share_to_send)
# TODO - add a more robust sanity check to ensure shares were sent.
print("Let's see what the first three users got:")
for i in range(3):
    print(f"User_{i} received:")
    print(*each_users_received_shares[i], sep="\n")
    print("\n")

# sanity check
assert len(each_users_received_shares[0]) == NUMBER_OF_USERS - 1

Distributing shares...

Let's see what the first three users got:
User_0 received:
tensor([[ 0.2255],
        [-1.9013]])
tensor([[-0.4681],
        [ 1.6676]])


User_1 received:
tensor([[6.8884],
        [5.1591]])
tensor([[8.1623],
        [0.0937]])


User_2 received:
tensor([[-1.5886],
        [-4.8217]])
tensor([[ 5.6760],
        [-3.9337]])




### Obfuscated Vector Generation

Each user can now calculate their own ofuscated feature vector. 

In [4]:
"""
GENERATING OBFUSCATED FEATURE Vectors:
"""
each_users_masked_vector = {}

for user in range(NUMBER_OF_USERS):
    sum_of_received_shares = torch.zeros(VECTOR_SHAPE)
    for share in each_users_received_shares[user]:
        sum_of_received_shares.add_(share)
    each_users_masked_vector[user] = each_users_Nth_share[user] + sum_of_received_shares 

print("Here's what each user will send to the aggregator:\n")
for user, masked_vector in each_users_masked_vector.items():
    print(f"User {user}:")
    print(masked_vector)
    print("\n")


Here's what each user will send to the aggregator:

User 0:
tensor([[-5.0462],
        [ 0.1971]])


User 1:
tensor([[ 9.2377],
        [11.2199]])


User 2:
tensor([[-3.2994],
        [-9.8827]])




### Aggregating Feature Vectors

Now we will mock an aggregator who receives each feature vector and computes a target function. In the actual protocol, each user would 'send' their masked vector to an aggregator. For simplicity's sake, we will simply act as an aggregator that has already received these vectors, and simply compute the target function.

In [5]:
# Computing Target Function: Aggregate Sum

target_result = sum(raw_feature_vectors[user] for user in raw_feature_vectors.keys())

masked_vector_result = sum(each_users_masked_vector[user] for user in each_users_masked_vector.keys())

print(f"Sum of masked vectors: \n\n {masked_vector_result}\n")
print(f"Sum of raw vectors: \n\n {target_result}")


Sum of masked vectors: 

 tensor([[0.8922],
        [1.5343]])

Sum of raw vectors: 

 tensor([[0.8922],
        [1.5343]])


### Utility of Protocol

As the output from the code block above shows, the aggregate sum of the masked vectors is equivalent to the aggregate sum of the raw vectors of the users. 

Also, it follows from this that simple mean of both vector sets are equal as well. 

SAFE thus does allow one to compute some aggregate functions over user data without having direct access to the user data. 

However, let us now consider how robustly it masks the raw feature vectors from potential adversaries who would try to infer things about the raw data from it.

### Evaluating SAFE's Robustness

We will consider this in terms of the information flow that the protocol allows from, and from the perspective of an adversarial aggregator, and an adversarial set of user(s). 

#### Adversarial Aggregator

We will assume that the aggregator knows what is presented plainly in Huth and Chuartwal's paper i.e., the equations listed in Section 4. Moreover, as described in the Protocol, the aggregator receives from each user an obfuscated feature vector. Finally, let us assume that the aggregator knows the values of the interval [-D, D] from which the users compute their share vectors. 

We want to ascertain what such an aggregator could infer from what they know. 

<!--Let us consider an even simpler example than the one we ran above: each user's raw feature vector is just one dimension.

Let `x` represent the `user_x`'s raw vector. Recall that, the value of a user's feature vectors are within the interval `[0,1]`, representing probabilities. So, let the value in `x` = 0.3. 

The aggregator knows the interval the users draws from is, as in our toy run,  `[-10, 10]`. 

Now, suppose the aggregator draws a random share from this interval, `y` , and `y` = -5.7.

Additionally, the aggregator receives an obfuscated vector `x'` from `user_x` , and `x'` = -5.4.-->

Suppose the adversarial aggregator drew their own N-1 shares from the same `[-D, D]` distribution. They could try to make an educated guess about the distribution of the raw feature values through reverse engineering:


In [10]:
# Aggregator samples (N-1) Random shares for each user.

aggregators_random_shares_per_user = {}
for user in range(NUMBER_OF_USERS):
    aggregators_random_shares_per_user[user] = []
    for i in range(NUMBER_OF_USERS - 1):
        share = [random.uniform(-D_VALUE, D_VALUE) for i in range(NUMBER_OF_FEATURES_PER_USER_VECTOR)]
        share = torch.reshape(torch.tensor(share_to_distribute), VECTOR_SHAPE)
        aggregators_random_shares_per_user[user].append(share)

received_masked_vectors = each_users_masked_vector

# Minus the Sum of the N-1 Shares From The Masked Vectors To Approximate Nth Share

approximation_of_each_users_Nth_share = {}

for user in range(NUMBER_OF_USERS):
    sum_of_aggregators_random_shares = torch.zeros(VECTOR_SHAPE)
    for v in aggregators_random_shares_per_user[user]:
        sum_of_aggregators_random_shares.add_(v)
    approximation_of_each_users_Nth_share[user] =  received_masked_vectors[user] - sum_of_aggregators_random_shares

print(approximation_of_each_users_Nth_share.items()) 

# Minus the Sum of the N-1 Shares Again From Approximation : TODO why?


dict_items([(0, tensor([[-0.3114],
        [-9.7826]])), (1, tensor([[13.9725],
        [ 1.2402]])), (2, tensor([[  1.4354],
        [-19.8625]]))])


/tmp/ipykernel_38923/2521218717.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  share = torch.reshape(torch.tensor(share_to_distribute), VECTOR_SHAPE)


### Making Protocol More Secure

Here, we illustrate the use of Homomorphic Encryption to make the protocol even more secure.

In [7]:
"""
Adjusting Feature Value Representation
"""

'\nAdjusting Feature Value Representation\n'